In [47]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [8]:
folder_path=glob("../../data/ddh_standard_frame_data/*")


def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df

In [46]:
Train_standard_file_path=[]
Train_not_standard_file_path=[]
Test_standard_file_path=[]
Test_not_standard_file_path=[]
Val_standard_file_path=[]
Val_not_standard_file_path=[]
for i in range(20):
    data_img_path=glob(folder_path[0]+"/data/*.jpg")
    data_img_path.sort()
    with open(folder_path[0]+'/annotations.json') as f:
        annotations_json = json.load(f)
    std_frame=AnnotationsPd(annotations_json[0])['frame']
    for frame in std_frame:
        file= data_img_path[frame]
        Train_standard_file_path.append(file)
    Train_not_standard_file_path=list(set(data_img_path)-set(Train_standard_file_path))   
    Train_standard_file_path=set(Train_standard_file_path)
    

AttributeError: 'set' object has no attribute 'append'

In [ ]:


with open(folder_path[2]+'/annotations.json') as f:
    annotations_json = json.load(f)   
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data2_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])

with open(folder_path[1]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in test_data_img_path if str(frame).zfill(5) in s] 
    Test_file_path.append(file[0])
    
with open(folder_path[3]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data4_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])
Train_file_path=list(set(Train_file_path))
Test_file_path=list(set(Test_file_path))

In [ ]:
Train_standard_file=[]
Train_TF_standard=[]
Test_standard_file=[]
Test_TF_standard=[]
for i in range(len(Train_file_path)):
    Train_TF_standard.append(1)
    shutil.copy(Train_file_path[i],'../../data/original_dataset/StandardFrame/Train/'+str(i)+'.jpg')
    Train_standard_file.append('StandardFrame/Train/'+str(i)+'.jpg')   
for i in range(len(Test_file_path)):
    Test_TF_standard.append(1)
    shutil.copy(Test_file_path[i],'../../data/original_dataset/StandardFrame/Test/'+str(i)+'.jpg')
    Test_standard_file.append('StandardFrame/Test/'+str(i)+'.jpg') 

In [ ]:
Train_not_standard_file1=list(set(data1_img_path)-set(Train_file_path))
Test_not_standard_file1=list(set(test_data_img_path)-set(Test_file_path))

for i in range(len(Train_not_standard_file1)):
    Train_TF_standard.append(0)
    shutil.copy(Train_not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/Train/'+str(i)+'.jpg')
    Train_standard_file.append('NotStandardFrame/Train/'+str(i)+'.jpg')
for i in range(len(Test_not_standard_file1)):
    Test_TF_standard.append(0)
    shutil.copy(Test_not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/Test/'+str(i)+'.jpg')
    Test_standard_file.append('NotStandardFrame/Test/'+str(i)+'.jpg')

In [ ]:
Train_original_standard_frame = pd.DataFrame({'file_path':Train_standard_file,
                             'standard_frame':Train_TF_standard})
Test_original_standard_frame = pd.DataFrame({'file_path':Test_standard_file,
                             'standard_frame':Test_TF_standard})
Train_original_standard_frame=shuffle(Train_original_standard_frame)
Test_original_standard_frame=shuffle(Test_original_standard_frame)
Test_original_standard_frame


In [ ]:
Train_original_standard_frame.to_csv('../../data/original_dataset/Train_data_set.csv', index = None) #csv파일로 생성
Test_original_standard_frame.to_csv('../../data/original_dataset/Test_data_set.csv', index = None) #csv파일로 생성